In [1]:
file_name = "Ground Truth Collection COMACO 2025"
local_path = f"/Users/GavilanIA/Downloads/COMACO/"
partner = "COMACO"
country = "zambia"

In [2]:
import pandas as pd
import openpyxl
import shapely
from typing import Callable, Optional, Union
import json
import dataclasses as dc
from shapely.lib import ShapelyError
import math
import numpy as np
from datetime import datetime, date



In [3]:
plots_df = pd.read_excel(
    f"{local_path}{file_name}.xlsx",
    sheet_name=0,
).rename(
    columns={
        "KEY": "PLOT_KEY",
    }
)
subplot_df = pd.read_excel(
    f"{local_path}{file_name}.xlsx",
    sheet_name=1,
).rename(
    columns={
        "PARENT_KEY": "PLOT_KEY",
        "KEY": "SUBPLOT_KEY"
    }
)
new_vegetation_df = pd.read_excel(
    f"{local_path}{file_name}.xlsx",
    sheet_name=2,
).rename(
    columns={
        "PARENT_KEY": "SUBPLOT_KEY",
        "KEY": "VEGETATION_KEY"
    }
)
measurement_df = pd.read_excel(
    f"{local_path}{file_name}.xlsx",
    sheet_name=3,
).rename(
    columns={
        "PARENT_KEY": "VEGETATION_KEY",
        "KEY": "MEASUREMENT_KEY"
    }
)
circumference_df = pd.read_excel(
    f"{local_path}{file_name}.xlsx",
    sheet_name=4,
).rename(
    columns={
        "PARENT_KEY": "MEASUREMENT_KEY",
        "KEY": "CIRCUMFERENCE_KEY"
    }
)


In [4]:
#Plot dataset
m_plots = pd.merge(plots_df,subplot_df, how="inner", on="PLOT_KEY") # only plots & subplots
m_plots["SubmissionDate"]= pd.to_datetime(m_plots["SubmissionDate"]).dt.date

#Define your period of interest
start_date = date(2025, 8, 11)
end_date = date(2025, 8, 28)

m_plots = m_plots[
    (m_plots["SubmissionDate"] >= start_date) &
    (m_plots["SubmissionDate"] <= end_date)
]

m_plots

,SubmissionDate,starttime,endtime,deviceid,devicephonenum,username,device_info,duration,caseid,enumerator,...,SET-OF-sub_plot_x,measured_subplots,instanceID,formdef_version,PLOT_KEY,gt_subplot,subplot_comments,SET-OF-new_vegetation,SUBPLOT_KEY,SET-OF-sub_plot_y
0,2025-08-12,2025-08-12 13:44:43.043,2025-08-12 18:01:24.576,2fbd34df95c52b75,NaN,ea@akvo.org,samsung|SM-A135F|14|SurveyCTO Collect 2.81.4 (...,11111,NaN,Mulenga Marvelous (314167),...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,16,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,2508111514,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,-11.9746915 31.2230352 1306.0 1.833;-11.974690...,It has so many obstacles,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...
1,2025-08-12,2025-08-12 13:44:43.043,2025-08-12 18:01:24.576,2fbd34df95c52b75,NaN,ea@akvo.org,samsung|SM-A135F|14|SurveyCTO Collect 2.81.4 (...,11111,NaN,Mulenga Marvelous (314167),...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,16,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,2508111514,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,-11.9746404 31.2230382 1307.0 3.0;-11.9746404 ...,The subplot had a lot of obstacles,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...
2,2025-08-12,2025-08-12 13:44:43.043,2025-08-12 18:01:24.576,2fbd34df95c52b75,NaN,ea@akvo.org,samsung|SM-A135F|14|SurveyCTO Collect 2.81.4 (...,11111,NaN,Mulenga Marvelous (314167),...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,16,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,2508111514,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,-11.9744558 31.2228638 1305.0 3.3;-11.9745084 ...,Lots of grass on the subplot,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...
3,2025-08-12,2025-08-12 13:44:43.043,2025-08-12 18:01:24.576,2fbd34df95c52b75,NaN,ea@akvo.org,samsung|SM-A135F|14|SurveyCTO Collect 2.81.4 (...,11111,NaN,Mulenga Marvelous (314167),...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,16,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,2508111514,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,-11.9746277 31.2228172 1305.0 3.5;-11.9746311 ...,The plot had a lot of obstacles,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...
4,2025-08-12,2025-08-12 13:44:43.043,2025-08-12 18:01:24.576,2fbd34df95c52b75,NaN,ea@akvo.org,samsung|SM-A135F|14|SurveyCTO Collect 2.81.4 (...,11111,NaN,Mulenga Marvelous (314167),...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,16,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,2508111514,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,-11.9748411 31.2227815 1302.0 4.25;-11.9748329...,NaN,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643,2025-08-23,2025-08-23 09:55:30.798,2025-08-23 15:23:07.647,f17d88bd6e102e04,NaN,ea@akvo.org,samsung|SM-A135F|14|SurveyCTO Collect 2.81.4 (...,5363,NaN,Chongo Mumba (654664),...,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,16,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0,2508191035,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0,-10.6424782 33.4614148 1238.0 1.7;-10.6424744 ...,NaN,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...
1644,2025-08-23,2025-08-23 09:55:30.798,2025-08-23 15:23:07.647,f17d88bd6e102e04,NaN,ea@akvo.org,samsung|SM-A135F|14|SurveyCTO Collect 2.81.4 (...,5363,NaN,Chongo Mumba (654664),...,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,16,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0,2508191035,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd3

In [5]:
#Vegetation datasets
m_veg = pd.merge(m_plots, new_vegetation_df, how= "inner", on ="SUBPLOT_KEY") # plots, species, nro of trees.
m_mea = pd.merge(m_veg,measurement_df, how="inner", on="VEGETATION_KEY" )
m_cir = pd.merge(m_mea, circumference_df, how="inner", on="MEASUREMENT_KEY")


Validation of tools

In [6]:
data_parameters= [
    "PLOT_KEY",
    "enumerator",
    "measured_subplots",
    "dbh_method",
    "dbh_unit",
    "height_method"]

data_table = m_plots[data_parameters]
data_table.drop_duplicates(subset="PLOT_KEY")

# Can you confirm that the dbh method, unit and height methods are correct?

,PLOT_KEY,enumerator,measured_subplots,dbh_method,dbh_unit,height_method
0,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232,Mulenga Marvelous (314167),16,circumference,cm,stick_method
16,uuid:0993e52a-51bc-45ee-ac14-3eb035045176,Chongo Mumba (654664),16,circumference,cm,stick_method
32,uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b,Abraham Msitini (168799),16,circumference,cm,stick_method
48,uuid:33e362a5-8621-48f8-9744-5c4a9a57f624,Chibale Gift (729957),16,circumference,cm,stick_method
64,uuid:efa69469-e110-4e06-bfee-82d72eea0601,Joseph Nkandu (336416),16,circumference,cm,stick_method
...,...,...,...,...,...,...
1568,uuid:53dd3cdf-eb3b-4cde-b4bf-431479d5b2a5,Penany manda (441963),16,circumference,cm,stick_method
1584,uuid:cf85da8c-2319-49f2-b029-5702aefd05e3,Chibale Gift (729957),16,circumference,cm,stick_method
1600,uuid:48a42b5d-e583-4c51-89b1-bd2b27a763de,Abraham Msitini (168799),16,circumference,cm,stick_method
1616,uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f,Joseph Nkandu (336416),16,circumference,cm,stick_method


To check how many subplots do not have vegetation and the reasons

In [7]:

col_missing_df = [
    "enumerator",
    "SUBPLOT_KEY",
    "subplot_comments"
]
subplots_veg = set(m_veg["SUBPLOT_KEY"].unique())
reference_subplots = set(m_plots["SUBPLOT_KEY"].unique())

missing_subplots = reference_subplots - subplots_veg

missing_df = m_plots[m_plots["SUBPLOT_KEY"].isin(missing_subplots)]

print("Number of missing subplots:", len(missing_subplots))

missing_df[col_missing_df]

Number of missing subplots: 114


,enumerator,SUBPLOT_KEY,subplot_comments
107,Abraham Msitini (168799),uuid:f29381db-a5d1-4b42-b235-a2249ca864f0/sub_...,Bare land
290,David Tembo (572397),uuid:bc44fc21-2c39-4ee1-8598-efb73ca25285/sub_...,Slope around the plot
291,David Tembo (572397),uuid:bc44fc21-2c39-4ee1-8598-efb73ca25285/sub_...,Obstacles around the subplot field
299,David Tembo (572397),uuid:bc44fc21-2c39-4ee1-8598-efb73ca25285/sub_...,Grasses was covered around the subplot field
425,Joel kumwenda (277523),uuid:67f08d53-99e8-4631-866b-d5e2f2a77253/sub_...,NaN
...,...,...,...
1537,George Salu (746183),uuid:c5d03ce3-d41d-4641-9c08-4dac92196bdc/sub_...,Bare land with many ridges in the subplot
1538,George Salu (746183),uuid:c5d03ce3-d41d-4641-9c08-4dac92196bdc/sub_...,Bare land with many ridges in the subplot
1539,George Salu (746183),uuid:c5d03ce3-d41d-4641-9c08-4dac92196bdc/sub_...,Bare land with many ridges in the subplot
1540,George Salu (746183),uuid:c5d03ce3-d41d-4641-9c08-4dac92196bdc/sub_...,Bare land with many ridges in the subplot


Check on density of trees and coverage in my subplots

In [24]:
#to check the density of trees
import plotly.express as px
density_parameters = [
    "enumerator",
    "SUBPLOT_KEY",
    "vegetation_type_number",
    "coverage_vegetation",
    "subplot_comments"
]


density = m_veg[density_parameters]

density_df = density.groupby("SUBPLOT_KEY").agg({
    "vegetation_type_number": "sum",
    "coverage_vegetation": "sum",
    "enumerator": "unique",
    "subplot_comments": "unique"}
).reset_index()

subplots_coverage = density_df[density_df["vegetation_type_number"]==0]

# fig_density = px.histogram(
#     density_df, 
#     x="vegetation_type_number",
#     nbins = 10,
#     labels={"vegetation_type_number":"trees in the subplot"},
#     title= "Distribution of trees in subplots"
#     )

# fig_density

density_df

,SUBPLOT_KEY,vegetation_type_number,coverage_vegetation,enumerator,subplot_comments
0,uuid:0028fd14-d27d-44f2-bdc7-42e985c82d09/sub_...,0.0,100.0,[Mulenga Marvelous (314167)],[nan]
1,uuid:0028fd14-d27d-44f2-bdc7-42e985c82d09/sub_...,0.0,100.0,[Mulenga Marvelous (314167)],[It had obstacles]
2,uuid:0028fd14-d27d-44f2-bdc7-42e985c82d09/sub_...,0.0,100.0,[Mulenga Marvelous (314167)],[It had obstacles]
3,uuid:0028fd14-d27d-44f2-bdc7-42e985c82d09/sub_...,0.0,100.0,[Mulenga Marvelous (314167)],[It had obstacles]
4,uuid:0028fd14-d27d-44f2-bdc7-42e985c82d09/sub_...,0.0,100.0,[Mulenga Marvelous (314167)],[nan]
...,...,...,...,...,...
1529,uuid:ffa02e12-7ee7-43cc-8e20-b8b64ba58e92/sub_...,0.0,25.0,[Abraham Msitini (168799)],[The plots is bare with a little bit of grass]
1530,uuid:ffa02e12-7ee7-43cc-8e20-b8b64ba58e92/sub_...,1.0,0.0,[Abraham Msitini (168799)],[Bare land]
1531,uuid:ffa02e12-7ee7-43cc-8e20-b8b64ba58e92/sub_...,0.0,50.0,[Abraham Msitini (168799)],[Bare land with covered trees]
1532,uuid:ffa02e12-7ee7-43cc-8e20-b8b64ba58e92/sub_...,0.0,25.0,[Abraham Msitini (168799)],[The subplot has covered trees]


Number of missing measurements (should match the number of subplots with only coverage), if not, enumerators are changing their answers

In [55]:
import pandas as pd

col_missing_veg = [
    "enumerator",
    "SUBPLOT_KEY",
    "subplot_comments",
    "crop_comments",
    "non_woody_species",
    "coverage_vegetation",
]

veg_mea = set(m_mea["SUBPLOT_KEY"].unique())
reference_veg = set(m_veg["SUBPLOT_KEY"].unique())

missing_veg = reference_veg - veg_mea

missing_veg_df = m_veg[m_veg["SUBPLOT_KEY"].isin(missing_veg)]

print("Number of subplots with only coverage:", len(subplots_coverage))
print("Number of missing measurements:", len(missing_veg))
missing_veg_df[col_missing_veg]


Number of subplots with only coverage: 663
Number of missing measurements: 661


,enumerator,SUBPLOT_KEY,subplot_comments,crop_comments,non_woody_species,coverage_vegetation
32,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,The plot had obstacles,NaN,poacea,100.0
35,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,NaN,NaN,poacea,75.0
38,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,It had obstacles,NaN,poacea,100.0
39,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,NaN,The grass is covering the whole subplot,poacea,100.0
64,Chongo Mumba (654664),uuid:0993e52a-51bc-45ee-ac14-3eb035045176/sub_...,NaN,Dry grass and many shrubs,poacea,100.0
...,...,...,...,...,...,...
3460,Chongo Mumba (654664),uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,NaN,NaN,poacea,100.0
3461,Chongo Mumba (654664),uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,NaN,NaN,poacea,100.0
3462,Chongo Mumba (654664),uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,NaN,NaN,poacea,100.0
3463,Chongo Mumba (654664),uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,NaN,NaN,poacea,100.0


Only when subplots 100% coverage do not match subplots missing measurements

In [56]:
same = ["SUBPLOT_KEY", "enumerator"]

cov = subplots_coverage[same]
measu = missing_veg_df[same]

veg_reverted = cov[~cov["SUBPLOT_KEY"].isin(measu["SUBPLOT_KEY"])]

veg_reverted

,SUBPLOT_KEY,enumerator
446,uuid:40904ad7-6c17-429d-ad2f-2b91dbf24f91/sub_...,[Joseph Nkandu (336416)]
803,uuid:7ceda479-b8a7-4a63-a8e2-a1a23a78a921/sub_...,[Mulenga Marvelous (314167)]


Tree_list - primary trees (quality check)

In [73]:
tree_list_parameters = [
        "enumerator",
        "gt_subplot",
        "vegetation_type_woody",
        "vegetation_species_type",
        "vegetation_type_youngtree",
        "vegetation_type_number",
        "vegetation_type_primary",
        "non_woody_species", 
        "woody_species", 
        "bamboo_species", 
        "banana_species", 
        "palm_species", 
        "other_species",
        "language_other_species",
        "coverage_vegetation",
        "PLOT_KEY",
        "SUBPLOT_KEY"
        ]


tree_list_primary = m_veg[tree_list_parameters]

trees_primary = tree_list_primary[
    (tree_list_primary["vegetation_type_primary"] == "yes_primary_group") &
    (tree_list_primary["woody_species"] == "other")
]

collector_primary_list = [
    "enumerator",
    "SUBPLOT_KEY",
    "other_species",
    "language_other_species",
    "vegetation_type_number"]

enumerator_trees_primary = trees_primary.filter(collector_primary_list).dropna(subset=["vegetation_type_number"]).dropna(subset=["other_species"])
enumerator_trees_primary


,enumerator,SUBPLOT_KEY,other_species,language_other_species,vegetation_type_number
0,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,Umucenja,Umucenja,1.0
1,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,Umufuka,Umufuka,58.0
3,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,Utuncindu,Utuncindu,1.0
4,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,Utumifuka,Utumifuka,15.0
6,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,Pawpaw,Pawpaw,1.0
...,...,...,...,...,...
3376,Chibale Gift (729957),uuid:cf85da8c-2319-49f2-b029-5702aefd05e3/sub_...,Muvondongo,Muvondongo,1.0
3425,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,Mumpagala,Mumpangala,1.0
3426,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,Mufuka,Mufuka,1.0
3441,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,Masuku,Masuku,2.0


Tree list - young trees

In [68]:
tree_list_young = m_veg[tree_list_parameters]

trees_young = tree_list_young[
    (tree_list_young["vegetation_type_youngtree"] == "yes_groupbelow1.3") &
    (tree_list_young["woody_species"] == "other")
]

collector_list_trees_young = [
    "enumerator",
    "SUBPLOT_KEY",
    "other_species",
    "language_other_species",
    "vegetation_type_number"]

enumerator_trees_young = trees_young.filter(collector_list_trees_young).dropna(subset=["vegetation_type_number"]).dropna(subset=["other_species"])
enumerator_trees_young

,enumerator,SUBPLOT_KEY,other_species,language_other_species,vegetation_type_number
59,Chongo Mumba (654664),uuid:0993e52a-51bc-45ee-ac14-3eb035045176/sub_...,Mukolo,Mukolo,7.0
89,Abraham Msitini (168799),uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b/sub_...,Chimufuka,Bemba,3.0
92,Abraham Msitini (168799),uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b/sub_...,Kamunengene,Bemba,2.0
93,Abraham Msitini (168799),uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b/sub_...,Uluputu,Bemba,15.0
94,Abraham Msitini (168799),uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b/sub_...,Kapulula mbuzi,Bemba,1.0
...,...,...,...,...,...
3331,Penany manda (441963),uuid:53dd3cdf-eb3b-4cde-b4bf-431479d5b2a5/sub_...,Mafutu,Mafutu,25.0
3343,Chibale Gift (729957),uuid:cf85da8c-2319-49f2-b029-5702aefd05e3/sub_...,Muombo,Muombo,2.0
3361,Chibale Gift (729957),uuid:cf85da8c-2319-49f2-b029-5702aefd05e3/sub_...,Nalenje,Nalenje,10.0
3367,Chibale Gift (729957),uuid:cf85da8c-2319-49f2-b029-5702aefd05e3/sub_...,Nalenje,Nalenje,17.0


Tree list - NO primary trees

In [69]:

tree_list = m_veg[tree_list_parameters]
trees = tree_list[
    (tree_list["vegetation_type_primary"] == "no") &
    (tree_list["woody_species"] == "other")
]

collector_list_trees = [
    "enumerator",
    "SUBPLOT_KEY",
    "other_species",
    "language_other_species",
    "vegetation_type_number"]

enumerator_trees = trees.filter(collector_list_trees).dropna(subset=["vegetation_type_number"]).dropna(subset=["other_species"])
percentage_trees = (enumerator_trees.vegetation_type_number.sum() / trees.vegetation_type_number.sum())*100
enumerator_trees

,enumerator,SUBPLOT_KEY,other_species,language_other_species,vegetation_type_number
88,Abraham Msitini (168799),uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b/sub_...,Umufungu- fungu,Bemba,1.0
104,Abraham Msitini (168799),uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b/sub_...,Nkololwe,Bemba,1.0
109,Abraham Msitini (168799),uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b/sub_...,Akaminwa,Bemba,1.0
110,Abraham Msitini (168799),uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b/sub_...,Chisaye,Bemba,2.0
111,Abraham Msitini (168799),uuid:1bf0b77f-79b5-4936-9fd4-88778b5d495b/sub_...,Mpundu,Bemba,1.0
...,...,...,...,...,...
3357,Chibale Gift (729957),uuid:cf85da8c-2319-49f2-b029-5702aefd05e3/sub_...,Umusokolobe,Umusokolobe,1.0
3359,Chibale Gift (729957),uuid:cf85da8c-2319-49f2-b029-5702aefd05e3/sub_...,Munyenjele,Munyenjele,1.0
3364,Chibale Gift (729957),uuid:cf85da8c-2319-49f2-b029-5702aefd05e3/sub_...,Munjenjele,Munyenjele,1.0
3368,Chibale Gift (729957),uuid:cf85da8c-2319-49f2-b029-5702aefd05e3/sub_...,Muuluka,Muuluka,1.0


Palm list check

In [ ]:

palm_list = m_veg[tree_list_parameters]


palms = palm_list[
    (palm_list["vegetation_species_type"] == "palms")
]

palms_list_trees = [
    "enumerator",
    "SUBPLOT_KEY",
    "vegetation_species_type",
    "other_species",
    "language_other_species",
    "vegetation_type_number"]



enumerator_palms = palms.filter(palms_list_trees).dropna(subset=["vegetation_type_number", "other_species"])
percentage_palms = (enumerator_palms.vegetation_type_number.sum() / palms.vegetation_type_number.sum())*100
print(f"Percentage of other species in tree group {percentage_palms}")

enumerator_palms

# As quality check for them a maximum of 5% for other species


Percentage of other species in tree group 0.0


,enumerator,SUBPLOT_KEY,vegetation_species_type,other_species,language_other_species,vegetation_type_number


Bamboo list check

In [76]:

bamboo_list = m_veg[tree_list_parameters]


bamboo = bamboo_list[
    (bamboo_list["vegetation_species_type"] == "bamboo")
]

bamboo_list_trees = [
    "enumerator",
    "SUBPLOT_KEY",
    "vegetation_species_type",
    "other_species",
    "language_other_species",
    "vegetation_type_number"]



enumerator_bamboo = bamboo.filter(bamboo_list_trees).dropna(subset=["vegetation_type_number", "other_species"])
percentage_bamboo = (enumerator_bamboo.vegetation_type_number.sum() / bamboo.vegetation_type_number.sum())*100
print(f"Percentage of other species in tree group {percentage_bamboo}")

enumerator_bamboo

# As quality check for them a maximum of 5% for other species


Percentage of other species in tree group nan


C:\Users\GavilanIA\AppData\Local\Temp\ipykernel_25132\1822327542.py:19: RuntimeWarning:

invalid value encountered in scalar divide



,enumerator,SUBPLOT_KEY,vegetation_species_type,other_species,language_other_species,vegetation_type_number


Coverage_quality_check

In [77]:
#Coverage quantification is allowed ONLY for non-woody species such as:
# grasses, weeds, or crops with short life spams such as rice, maize, potatoes. 
# The impact of adding species from other categories is very high because we cannot use these values for biomass.


coverage = tree_list[
    (tree_list["vegetation_type_woody"] == "nonwoody_coverage")|
    (tree_list["vegetation_type_youngtree"] == "no_coverage")
]

collector_list_coverage = [
    "enumerator",
    "SUBPLOT_KEY",
    "other_species",
    "language_other_species",
    "coverage_vegetation"]

enumerator_coverage = coverage.filter(collector_list_coverage).dropna(subset=["other_species"])

percentage_cov = (enumerator_coverage.shape[0] / coverage.shape[0])*100
print(f"Percentage of other species in coverage {percentage_cov}")

enumerator_coverage
# enumerator_coverage.to_excel("coverage_list.xlsx", index=False)

Percentage of other species in coverage 9.400123685837972


,enumerator,SUBPLOT_KEY,other_species,language_other_species,coverage_vegetation
143,Chibale Gift (729957),uuid:33e362a5-8621-48f8-9744-5c4a9a57f624/sub_...,Umunga,Umunga,25.0
145,Chibale Gift (729957),uuid:33e362a5-8621-48f8-9744-5c4a9a57f624/sub_...,Umunga,Umunga,25.0
162,Joseph Nkandu (336416),uuid:efa69469-e110-4e06-bfee-82d72eea0601/sub_...,Grass,Grass,75.0
165,Joseph Nkandu (336416),uuid:efa69469-e110-4e06-bfee-82d72eea0601/sub_...,Grass,Grass,75.0
169,Joseph Nkandu (336416),uuid:efa69469-e110-4e06-bfee-82d72eea0601/sub_...,Grass,Grass,50.0
...,...,...,...,...,...
2965,Abraham Msitini (168799),uuid:062c52d6-c494-463b-8421-9b9b2217ddef/sub_...,Chisega,Tumbuka,25.0
2974,Abraham Msitini (168799),uuid:062c52d6-c494-463b-8421-9b9b2217ddef/sub_...,Butuzi,Tumbuka,75.0
3008,Abraham Msitini (168799),uuid:062c52d6-c494-463b-8421-9b9b2217ddef/sub_...,Fisaye,Bemba,25.0
3036,Joseph Nkandu (336416),uuid:9dca4fe2-6cb7-422b-923b-36e950268cd3/sub_...,Tobacco,Tobbaco,50.0


Vegetation quality checks

In [15]:
import statistics
import plotly.express as px
import numpy as np

# to check homogeneity in height of groups

veg_parameters = [
    "enumerator",
    "VEGETATION_KEY",
    "SUBPLOT_KEY",
    "vegetation_type_number",
    "tree_height_m",
    "tree_year_planted",
    "tree_prune",
    "tree_coppiced",
]

height_check = m_mea[veg_parameters]

median_check = height_check.groupby("VEGETATION_KEY")["tree_height_m"].median().reset_index(name="median_height")
height_total = pd.merge(height_check, median_check, how="inner", on = "VEGETATION_KEY")
height_total["Upper_outliers"] = height_total.apply(lambda row: "outlier" if row["tree_height_m"] > (row["median_height"]*4) else "ok", axis=1)
height_total["Lower_outliers"] = height_total.apply(lambda row: "outlier" if row["tree_height_m"] < (row["median_height"]/4) else "ok", axis=1)
# this script will show the outliers (height up & down) within a group of trees, important to check whether there are no pruning or coppicing/pollarding practices that could be influencing the result
height_total


,enumerator,VEGETATION_KEY,SUBPLOT_KEY,vegetation_type_number,tree_height_m,tree_year_planted,tree_prune,tree_coppiced,median_height,Upper_outliers,Lower_outliers
0,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,1.0,20.0,1970-01-01,no,no,20.0,ok,ok
1,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,58.0,2.5,2018-01-01,no,no,2.5,ok,ok
2,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,58.0,1.8,2018-01-01,no,no,2.5,ok,ok
3,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,58.0,2.8,2018-01-01,no,no,2.5,ok,ok
4,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,58.0,2.5,2018-01-01,no,no,2.5,ok,ok
...,...,...,...,...,...,...,...,...,...,...,...
3432,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,1.0,4.7,2021-01-01,no,no,3.5,ok,ok
3433,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,1.0,3.1,2021-01-01,no,no,3.5,ok,ok
3434,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,1.0,3.5,2021-01-01,no,no,3.5,ok,ok
3435,Chongo Mumba (654664),uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,1.0,7.3,2001-01-01,no,no,7.3,ok,ok


Checking tall trees

In [17]:
# Here it is important to check that tall trees are realistic with the planting age (e.g. trees 34 meters tall should not be planted in 2025)

check = height_total[height_total["tree_height_m"]> 25] #what should be the threshold?
check
# check.to_excel("height_super_tall.xlsx", index=False)

,enumerator,VEGETATION_KEY,SUBPLOT_KEY,vegetation_type_number,tree_height_m,tree_year_planted,tree_prune,tree_coppiced,median_height,Upper_outliers,Lower_outliers
553,Chongo Mumba (654664),uuid:1d76722b-5bec-4d4a-a3a4-9cca02e1281e/sub_...,uuid:1d76722b-5bec-4d4a-a3a4-9cca02e1281e/sub_...,87.0,51.0,2016-01-01,no,no,4.0,outlier,ok
2241,Chibale Gift (729957),uuid:01b1118d-3748-4a5f-a9e6-ac26121f86ab/sub_...,uuid:01b1118d-3748-4a5f-a9e6-ac26121f86ab/sub_...,1.0,34.1,2025-01-01,no,no,34.1,ok,ok
2796,Joel kumwenda (277523),uuid:cd18c1f1-de26-4224-8855-8c1b3115f2bd/sub_...,uuid:cd18c1f1-de26-4224-8855-8c1b3115f2bd/sub_...,2.0,71.0,2008-01-01,no,no,39.1,ok,ok


Check on coppicing and pollarding practices

In [16]:
coppice_parameters = [
    "enumerator",
    "VEGETATION_KEY",
    "vegetation_type_number",
    "tree_height_m",
    "prune_height",
    "coppiced_height",
    "tree_year_planted",
    "tree_prune",
    "tree_coppiced",
]

coppice_check = m_mea[coppice_parameters]


median_cop_check = coppice_check.groupby("VEGETATION_KEY")["coppiced_height"].median().reset_index(name="median_coppiced")

coppiced_total = pd.merge(coppice_check, median_cop_check, how="inner", on = "VEGETATION_KEY")
coppiced_total["Upper_outliers"] = coppiced_total.apply(lambda row: "outlier" if row["coppiced_height"] > (row["median_coppiced"]*4) else "ok", axis=1)
coppiced_total["Lower_outliers"] = coppiced_total.apply(lambda row: "outlier" if row["coppiced_height"] < (row["median_coppiced"]/4) else "ok", axis=1)
# to check whether the prune height is higher than the coppiced height
coppiced_total

,enumerator,VEGETATION_KEY,vegetation_type_number,tree_height_m,prune_height,coppiced_height,tree_year_planted,tree_prune,tree_coppiced,median_coppiced,Upper_outliers,Lower_outliers
0,NYIMBILI CHRISPINE (183568),uuid:43fe2383-8958-472c-8025-69887d3b58e1/sub_...,1.0,3.1,NaN,NaN,2020-01-01,no,no,NaN,ok,ok
1,NYIMBILI CHRISPINE (183568),uuid:43fe2383-8958-472c-8025-69887d3b58e1/sub_...,1.0,4.0,NaN,NaN,2016-01-01,no,no,NaN,ok,ok
2,NYIMBILI CHRISPINE (183568),uuid:43fe2383-8958-472c-8025-69887d3b58e1/sub_...,1.0,4.5,NaN,NaN,2016-01-01,yes,yes,NaN,ok,ok
3,NYIMBILI CHRISPINE (183568),uuid:43fe2383-8958-472c-8025-69887d3b58e1/sub_...,1.0,4.2,NaN,NaN,1998-01-01,no,no,NaN,ok,ok
4,NYIMBILI CHRISPINE (183568),uuid:43fe2383-8958-472c-8025-69887d3b58e1/sub_...,1.0,1.5,NaN,NaN,2020-01-01,no,no,NaN,ok,ok
...,...,...,...,...,...,...,...,...,...,...,...,...
1546,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,1.0,4.7,NaN,NaN,2021-01-01,no,no,NaN,ok,ok
1547,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,1.0,3.1,NaN,NaN,2021-01-01,no,no,NaN,ok,ok
1548,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,1.0,3.5,NaN,NaN,2021-01-01,no,no,NaN,ok,ok
1549,Chongo Mumba (654664),uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,1.0,7.3,NaN,NaN,2001-01-01,no,no,NaN,ok,ok


Check on number of stems

In [17]:
stems_parameters = [
    "enumerator",
    "VEGETATION_KEY",
    "other_species",
    "woody_species",
    "banana_species",
    "vegetation_type_number",
    "nr_stems_bh",
    "nr_stems_10cm",
    "tree_year_planted",
]

stems = m_mea[stems_parameters]

stems[stems["nr_stems_bh"]>20] #Should be more than 40, should I constrain it?

,enumerator,VEGETATION_KEY,other_species,woody_species,banana_species,vegetation_type_number,nr_stems_bh,nr_stems_10cm,tree_year_planted
209,Penany manda (441963),uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,NaN,NaN,musa_acuminata,1.0,25.0,1.0,2016-01-01
213,Penany manda (441963),uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,NaN,NaN,musa_acuminata,1.0,25.0,1.0,2016-01-01
244,NYIMBILI CHRISPINE (183568),uuid:888125b0-47e7-4fcd-b79b-c02b6a819e39/sub_...,Kasutho,other,NaN,1.0,24.0,1.0,2024-01-01
279,Joel kumwenda (277523),uuid:6bfab88c-3983-40f6-b7a4-55e3baa0479c/sub_...,NaN,parinari_curatellifolia,NaN,1.0,24.0,20.0,2021-01-01
367,Chibale Gift (729957),uuid:01b1118d-3748-4a5f-a9e6-ac26121f86ab/sub_...,NaN,gliricidia_sepium,NaN,1.0,26.0,1.0,2014-01-01
370,Chibale Gift (729957),uuid:01b1118d-3748-4a5f-a9e6-ac26121f86ab/sub_...,NaN,gliricidia_sepium,NaN,7.0,21.0,2.0,2014-01-01
378,Chibale Gift (729957),uuid:01b1118d-3748-4a5f-a9e6-ac26121f86ab/sub_...,NaN,parinari_curatellifolia,NaN,1.0,26.0,1.0,2021-01-01
442,Abraham Msitini (168799),uuid:c1c6e3ee-b220-4357-89c2-8c01fa854e84/sub_...,NaN,mangifera_indica,NaN,1.0,21.0,3.0,2014-01-01
547,Joel kumwenda (277523),uuid:d3dd11bd-7b0c-4420-a545-f6ae880c8475/sub_...,Maruben,other,NaN,1.0,21.0,11.0,2018-01-01
552,Joel kumwenda (277523),uuid:d3dd11bd-7b0c-4420-a545-f6ae880c8475/sub_...,NaN,kigelia_africana,NaN,1.0,23.0,10.0,2023-01-01


Circumference check

In [ ]:
#to discuss the circumference of the groups

circumference = [
"enumerator",
"VEGETATION_KEY",
"MEASUREMENT_KEY",
"vegetation_type_number",
"circumference_bh",
"circumference_10cm",
"tree_year_planted"
]

circumference_list = m_cir[circumference]
median_cir = circumference_list.groupby("MEASUREMENT_KEY")["circumference_bh"].median().reset_index(name="median_cir")
cir_total = pd.merge(circumference_list, median_cir, how="inner", on = "MEASUREMENT_KEY")
cir_total["Upper_outliers"] = cir_total.apply(lambda row: "outlier" if row["circumference_bh"] > (row["median_cir"]*4) else "ok", axis=1)
cir_total["Lower_outliers"] = cir_total.apply(lambda row: "outlier" if row["circumference_bh"] < (row["median_cir"]/4) else "ok", axis=1)
cir_total




,enumerator,VEGETATION_KEY,MEASUREMENT_KEY,vegetation_type_number,circumference_bh,circumference_10cm,tree_year_planted,median_cir,Upper_outliers,Lower_outliers
0,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,1.0,170.0,NaN,1970-01-01,170.0,ok,ok
1,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,58.0,15.0,NaN,2018-01-01,15.0,ok,ok
2,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,58.0,10.0,NaN,2018-01-01,10.0,ok,ok
3,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,58.0,17.0,NaN,2018-01-01,11.0,ok,ok
4,Mulenga Marvelous (314167),uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,uuid:96890fa1-07c6-4d08-ab58-ab2e8c080232/sub_...,58.0,5.0,NaN,2018-01-01,11.0,ok,ok
...,...,...,...,...,...,...,...,...,...,...
5392,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,1.0,NaN,500.0,2021-01-01,NaN,ok,ok
5393,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,1.0,30.0,NaN,2021-01-01,30.0,ok,ok
5394,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,1.0,30.0,NaN,2021-01-01,30.0,ok,ok
5395,Chongo Mumba (654664),uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,uuid:9681b48e-9bdd-47f0-96d9-2caa60cd37e0/sub_...,1.0,205.0,NaN,2001-01-01,205.0,ok,ok


In [19]:
cir_total[cir_total["circumference_bh"]> 90] #what should be the threshold?

#Here we should focus on relations btween tree age and circumference to avoid errors (e.g. circumference 300 and platn age 2024)


,enumerator,VEGETATION_KEY,MEASUREMENT_KEY,vegetation_type_number,circumference_bh,circumference_10cm,tree_year_planted,median_cir,Upper_outliers,Lower_outliers
23,Joel kumwenda (277523),uuid:261b0abe-494c-4dc7-b4ed-17ca84e3a79a/sub_...,uuid:261b0abe-494c-4dc7-b4ed-17ca84e3a79a/sub_...,2.0,97.5,NaN,1992-01-01,97.5,ok,ok
47,David Tembo (572397),uuid:5d28eea5-4359-489e-8dad-9e765baa003b/sub_...,uuid:5d28eea5-4359-489e-8dad-9e765baa003b/sub_...,3.0,195.0,NaN,2000-01-01,195.0,ok,ok
48,David Tembo (572397),uuid:5d28eea5-4359-489e-8dad-9e765baa003b/sub_...,uuid:5d28eea5-4359-489e-8dad-9e765baa003b/sub_...,3.0,98.0,NaN,2000-01-01,82.3,ok,ok
55,David Tembo (572397),uuid:5d28eea5-4359-489e-8dad-9e765baa003b/sub_...,uuid:5d28eea5-4359-489e-8dad-9e765baa003b/sub_...,1.0,99.0,NaN,2000-01-01,80.0,ok,ok
59,David Tembo (572397),uuid:5d28eea5-4359-489e-8dad-9e765baa003b/sub_...,uuid:5d28eea5-4359-489e-8dad-9e765baa003b/sub_...,1.0,93.0,NaN,2000-01-01,80.0,ok,ok
...,...,...,...,...,...,...,...,...,...,...
2151,Abraham Msitini (168799),uuid:48a42b5d-e583-4c51-89b1-bd2b27a763de/sub_...,uuid:48a42b5d-e583-4c51-89b1-bd2b27a763de/sub_...,1.0,97.0,NaN,2003-01-01,105.5,ok,ok
2152,Abraham Msitini (168799),uuid:48a42b5d-e583-4c51-89b1-bd2b27a763de/sub_...,uuid:48a42b5d-e583-4c51-89b1-bd2b27a763de/sub_...,1.0,122.0,NaN,2002-01-01,122.0,ok,ok
2159,Abraham Msitini (168799),uuid:48a42b5d-e583-4c51-89b1-bd2b27a763de/sub_...,uuid:48a42b5d-e583-4c51-89b1-bd2b27a763de/sub_...,1.0,503.0,NaN,2013-01-01,503.0,ok,ok
2184,Joseph Nkandu (336416),uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,uuid:59e94219-a79a-41a1-a106-8ba4bd293e2f/sub_...,2.0,109.0,NaN,2000-01-01,109.0,ok,ok


In [20]:
cir_total[cir_total["circumference_10cm"]> 90] #what should be the threshold?

#Here we should focus on relations btween tree age and circumference to avoid errors (e.g. circumference 300 and platn age 2024)

,enumerator,VEGETATION_KEY,MEASUREMENT_KEY,vegetation_type_number,circumference_bh,circumference_10cm,tree_year_planted,median_cir,Upper_outliers,Lower_outliers
0,NYIMBILI CHRISPINE (183568),uuid:43fe2383-8958-472c-8025-69887d3b58e1/sub_...,uuid:43fe2383-8958-472c-8025-69887d3b58e1/sub_...,1.0,NaN,510.0,2020-01-01,NaN,ok,ok
6,NYIMBILI CHRISPINE (183568),uuid:43fe2383-8958-472c-8025-69887d3b58e1/sub_...,uuid:43fe2383-8958-472c-8025-69887d3b58e1/sub_...,1.0,NaN,350.0,2020-01-01,NaN,ok,ok
99,Mulenga Marvelous (314167),uuid:42129e84-a5db-4062-8ea7-3b5d2a721532/sub_...,uuid:42129e84-a5db-4062-8ea7-3b5d2a721532/sub_...,1.0,NaN,124.0,2017-01-01,NaN,ok,ok
162,Chibale Gift (729957),uuid:d52e8f44-5c76-4cbe-8660-577893b47823/sub_...,uuid:d52e8f44-5c76-4cbe-8660-577893b47823/sub_...,1.0,NaN,130.0,1979-01-01,NaN,ok,ok
164,Chibale Gift (729957),uuid:d52e8f44-5c76-4cbe-8660-577893b47823/sub_...,uuid:d52e8f44-5c76-4cbe-8660-577893b47823/sub_...,1.0,NaN,270.0,1979-01-01,NaN,ok,ok
256,Penany manda (441963),uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,3.0,NaN,400.0,2018-01-01,NaN,ok,ok
257,Penany manda (441963),uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,3.0,NaN,350.0,2018-01-01,NaN,ok,ok
258,Penany manda (441963),uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,3.0,NaN,340.0,2018-01-01,NaN,ok,ok
268,Penany manda (441963),uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,1.0,NaN,370.0,2016-01-01,NaN,ok,ok
269,Penany manda (441963),uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,uuid:d4146c08-5171-4677-ab76-9ddbfe3b6e23/sub_...,1.0,NaN,410.0,2016-01-01,NaN,ok,ok


In [ ]:
dd= m_veg.groupby("SUBPLOT_KEY")["vegetation_type_number"].sum().reset_index(name="total_vegetation")

other_vegetation = m_veg[m_veg['other_species'].str.lower() == 'other'].groupby('SUBPLOT_KEY')['vegetation_number'].sum().reset_index(name='other_vegetation')


KeyError: 'other_species'

In [ ]:
m_cir.columns

final_list = [
    "SUBPLOT_KEY",
    "enumerator",
    "vegetation_type_number",
    "tree_year_planted",
    "non_woody_species",
    "woody_species",
    "bamboo_species",
    "banana_species",
    "palm_species",
    "coverage_vegetation",
    "coverage_height",
    "tree_height_m",
    "tree_prune",
    "prune_height",
    "coppiced_height",
    "nr_stems_bh",
    "nr_stems_10cm",
    "circumference_bh_group_count",
    ""

]

# enumerator_final = m_cir[final_list]

# enumerator_final

Index(['SubmissionDate', 'starttime', 'endtime', 'deviceid', 'devicephonenum',
       'username', 'device_info', 'duration', 'caseid', 'enumerator',
       'plot_id_acorn', 'enumerator_id', 'enumerator_name', 'gt_plot_nr',
       'dbh_method', 'dbh_unit', 'height_method', 'location', 'rainfall',
       'season', 'year_agroforestry', 'terrain_difficulties',
       'terrain_difficulties_none', 'terrain_difficulties_plot_too_small',
       'terrain_difficulties_steep_slope',
       'terrain_difficulties_farmer_notinformed',
       'terrain_difficulties_no_trees_bare_land',
       'terrain_difficulties_plot_different',
       'terrain_difficulties_plot_access', 'terrain_difficulties_other',
       'comments', 'terrain_difficulties_other.1', 'gt_plot', 'sub_plot_count',
       'SET-OF-sub_plot_x', 'measured_subplots', 'instanceID',
       'formdef_version', 'PLOT_KEY', 'gt_subplot', 'subplot_comments',
       'SET-OF-new_vegetation_x', 'SUBPLOT_KEY', 'SET-OF-sub_plot_y',
       'vegetation_